# Introduction to data masking with pyActigraphy

This introduction notebook illustrates how to mask potentially spurious inactivity periods.

First, load the pyActigraphy package:

In [ ]:
import pyActigraphy

As well as your favourite plotting package:

In [ ]:
import plotly.graph_objs as go

## Read an individual actigraphy file

As an example, let’s read a .awd file located in the test directory of the pyActigraphy package:

In [ ]:
import os
fpath = os.path.join(os.path.dirname(pyActigraphy.__file__),'tests/data/')

In [ ]:
raw = pyActigraphy.io.read_raw_awd(fpath+'example_01_mask.AWD')

## Plot the data

In [ ]:
layout = go.Layout(title="Actigraphy data", xaxis=dict(title="Date time"), yaxis=dict(title="Counts/period"), showlegend=False)

In [ ]:
go.Figure(data=go.Scatter(x=raw.data.index.astype(str), y=raw.data), layout=layout)

At first sight, it seems that there is a suspicious period of total inactivity (activity counts equal to zero), starting at 9h30 on the 27th of January.

It seems spurious for several reasons:

* it happens outside the usually sleep time window (although a nap might be possible...);
* it consists of a long series of consecutive "total" inactivity (i.e. zero count) while other potential sleep periods do not;
* it exhibits a sharp decrease of activity with respect to the surrounding time periods;


Most likely, it corresponds to a period where the actigraph has been removed by the participant.

## Mask inactive data

While it is fairly (although not totally) simple to deal with inactivity periods at the beginning and at the end of the recordings (cf this other tutorial [notebook](https://ghammad.github.io/pyActigraphy/pyActigraphy-SSt-log.html)), it is more difficult to deal, in an automatic fashion, with inactivity periods happening during the recording. 

Fortunately, the **_pyActigraphy_** package offers a simple way to deal with these periods by masking the corresponding data.

### Definition

Spurious inactivity periods are defined as series of consecutive zeros. The lenght of these series is configurable.

### Create a mask automatically...

In [ ]:
raw.frequency

In [ ]:
# The duration corresponds either to the minimal number of inactive epochs (ex: duration=120)
# or the minimal length of the inactive period (duration='2h00min')
raw.create_inactivity_mask(duration='2h00min')

In [ ]:
raw.inactivity_length

**WARNING**: Creating a mask does not mean it is applied. Cf. next section.**

Before applying it, let's visualize it:

In [ ]:
layout = go.Layout(title="Data mask", xaxis=dict(title="Date time"), yaxis=dict(title="Mask"), showlegend=False)

In [ ]:
go.Figure(data=go.Scatter(x=raw.mask.index.astype(str),y=raw.mask),layout=layout)

Periods in the data for which the mask is equal to zero will be masked (once the mask is applied).

As expected, the spurious period of total inactivity of the 27th of January has been found by the **_pyActigraphy_** package and therefore can be masked upon request (cf. next section).

It is possible to dynamically change the configuration of the minimal duration. At each change, the mask is recomputed.

In [ ]:
raw.inactivity_length = '4h'

In [ ]:
go.Figure(data=go.Scatter(x=raw.mask.index.astype(str),y=raw.mask),layout=layout)

### ... or specify the periods manually.

First, let's reset the mask by setting its length to `None`:

In [ ]:
# Reset mask
raw.inactivity_length = None

In [ ]:
raw.mask

#### `add_mask_period`

The function `add_mask_period` allows users to explicitly specify the period that should be masked.

If no mask has been created before, it is created on the fly when the first mask period is specified.

This function can be used as many times as needed.

In [ ]:
raw.add_mask_period(start='1918-01-27 09:30:00',stop='1918-01-27 17:48:00')

Let's now inspect the newly created mask:

In [ ]:
go.Figure(data=go.Scatter(x=raw.mask.index.astype(str),y=raw.mask),layout=layout)

#### `add_mask_periods`

The `add_mask_periods` function allows users to use a file to specify several additional mask periods at once.

The file format accepted by **pyActigraphy** is the following:


| Mask | Start_time | Stop_time | Remarks (optional)
| --- | --- | --- | --- |
| mask_01 | 1918-02-01 20:42:00 | 1918-02-01 21:35:00| Reported in the sleep diary. |
| mask_02 | 1918-02-02 13:20:00 | 1918-02-02 14:14:00| Found by visual inspection. |

In [ ]:
raw.add_mask_periods(os.path.join(os.path.dirname(pyActigraphy.__file__),'tests/data/example_masklog.csv'))

In [ ]:
go.Figure(data=go.Scatter(x=raw.mask.index.astype(str),y=raw.mask),layout=layout)

It can be seen here that two new periods were added to the mask, in addition to the period previously specified manually.

### Apply the mask

Masking the spurious inactivity periods is crucial as it can substantially affect the calculation of the rest-activity rhythm related variables.

For example:

In [ ]:
raw.IS()

Applying the mask is controlled by the following variable:

In [ ]:
raw.mask_inactivity = True

In [ ]:
raw.IS()

Interesting, isn't it?

Besides, the application of the masking is dynamic:

In [ ]:
raw.mask_inactivity = False

In [ ]:
raw.IS()

A last word; be cautious when masking your data. Series of consecutive periods of total inactivity might happen during sleep... A visual check of the mask is always a good idea.